<a href="https://colab.research.google.com/github/williamzhao01123/crypto-lab/blob/master/UTXO_Project_Documentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Create Joint_All in Bigquery with SQL

## Step 1: Query data from the input and output public data sets

### inputs

In Bigquery, first choose destination table as



```
crypto-291811.UTXO.inputs
```



Then put the SQL code in Bigquery Query BOX. Only the variables of interest are selected in this step.


```
SELECT 
  spent_transaction_hash,
  spent_output_index,	
  block_timestamp AS spent_block_timestamp,   
FROM `bigquery-public-data.crypto_bitcoin.inputs`
```


###outputs

In Bigquery, first choose destination table as



```
crypto-291811.UTXO.outputs
```



Then put the SQL code in Bigquery Query BOX. Only the variables of interest are selected in this step.



```
SELECT 
  transaction_hash,
  block_timestamp,
  index,	
  value
FROM `bigquery-public-data.crypto_bitcoin.outputs`
```



## Step 2: Joint the two tables by transaction hash and index

Choose destination table as



```
crypto-291811.UTXO.joint
```



Put the SQL in Bigquery Query BOX



```
SELECT
  outputs.value,  
  outputs.transaction_hash,
  inputs.spent_transaction_hash,
  outputs.index,
  inputs.spent_output_index,
  outputs.block_timestamp,
  inputs.spent_block_timestamp,
FROM 
  `crypto-291811.UTXO.outputs` AS outputs
LEFT JOIN 
  `crypto-291811.UTXO.inputs` AS inputs
ON outputs.transaction_hash=inputs.spent_transaction_hash  
AND outputs.index = inputs.spent_output_index
```



For the above steps, you may also work in Google Colab. Reference the code here: https://github.com/sunshineluyao/UTXO/blob/main/UTXO_Age_Distribution_Data_queried_from_public_data_set.ipynb

## Step 3: Format Data and Keep only variables of interest

First choose destination table as



```
crypto-291811.UTXO.joint_all
```



Then put the SQL code in Bigquery Query BOX



```
SELECT 
  value,
  FORMAT_TIMESTAMP("%Y-%m-%d", block_timestamp) AS block_date,
  FORMAT_TIMESTAMP("%Y-%m-%d", spent_block_timestamp) AS spent_block_date,
FROM
  `crypto-291811.UTXO.joint`
```



## Step 4: Partition and Cluster the Table

After creating a data file that includes the information of value, block_date and spent_block_date, we would like to partition the table based on dates. This step will significantly save the cost of query in later data analysis work.

### Partitioning the data by born date

A new table is created in bigquery by running the following SQL code in Query Box. This step costs 45GB query. As at most 4000 partitions are allowed in Bigquery, only data after 2012 is partitioned. Data before 2012 is small enough to be processed locally and seperately.



```
CREATE TABLE
  `crypto-291811.UTXO.joint_all_partitionedbyborn12`
PARTITION BY
  DATE(block_timestamp) AS
SELECT
  *
WHERE
  block_timestamp > TIMESTAMP('2012-01-01 00:00:00+00')
FROM
  `crypto-291811.UTXO.joint_all`
```



### Partitioning the data by death date

Similarly, a second partitioned table is created with spent_block_date as the partition column.



```
CREATE TABLE
  `crypto-291811.UTXO.joint_all_partitionedbydeath12`
PARTITION BY
  DATE(spent_block_timestamp) AS
SELECT
  *
WHERE
  spent_block_timestamp > TIMESTAMP('2012-01-01 00:00:00+00')
FROM
  `crypto-291811.UTXO.joint_all`
```



An extra step that can be taken here is to further cluster the data by block_date. Clustering is another technique that could help save query cost in Bigquery. There are no limits on the number of clusters to be created, but the benefits clustering brings is much smaller than partitioning. For example, the SQL below creates a table partitioned by spent_block_date but clustered by block_date.



```
CREATE TABLE
  `crypto-291811.UTXO.joint_all_pdeath_cborn`
PARTITION BY
  DATE(spent_block_timestamp) 
  CLUSTER BY DATE(block_timestamp) AS
SELECT
  *
WHERE
  spent_block_timestamp > TIMESTAMP('2012-01-01 00:00:00+00')
FROM
  `crypto-291811.UTXO.joint_all`
```



# Processing the Data in Kaggle

With the data file joint_all ready in Bigquery, we process the UTXO data in kaggle to further refine the information. The kaggle project can be found at [UTXO Project](https://www.kaggle.com/williamzhao123/utxo-project-william-zhao).

##Step 1: Import Data into Kaggle

###From Big Query

Kaggle can read data by running a query in Bigquery if the account is authorized. An example program is shown below, though this notebook is not authorized to query data.

In [ ]:
PROJECT_ID = 'crypto-291811'

from datetime import date, timedelta
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location='US')
dataset_ref = client.dataset('UTXO', project=PROJECT_ID)
dataset = client.get_dataset(dataset_ref)
tables = list(client.list_tables(dataset))
# Print names of all tables in the dataset
for table in tables:  
    print(table.table_id)

start_date = date(2013,1,1)
end_date = start_date+timedelta(days=1)
query1 = """
      SELECT 
        *
      FROM 
        `crypto-291811.UTXO.joint_all_partitionedbyborn12`
      WHERE
        block_timestamp >= TIMESTAMP('""" + str(start_date) + """ 00:00:00+00')
      AND 
        block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')"""
query_job1 = client.query(query1)
# Make an API request  to run the query and return a pandas DataFrame
data = query_job1.to_dataframe()
data.head()

We feed the UTXO data after 2012 day by day to kaggle with the above approach. As the data is partitioned by block_date or spent_block_date, filtering data either block_date or spent_block_date is very efficient in Bigquery.

###From Google Drive

Data before 2012, on the other hand, is imported to kaggle from Google Drive. When we partitioned the data, only data after 2012 is kept in Bigquery in order to limit number of partitions to 4000. As the data before 2011 is small enough to use as a whole, we import the data file from Google Drive directly with the following code.

In [ ]:
!conda install -y gdown
import gdown
url = 'https://drive.google.com/uc?id=1-83ycaEkWjV0TsD7875RL9uRdXSIh9_D'
output = 'joint_2011.csv'
gdown.download(url, output, quiet=False)
data = pd.read_csv('joint_2011.csv')
data.head()

##Step 2: Processing Daily Data in Kaggle

While the data before 2012 and after 2012 are imported seperately into kaggle, data are analyzed in the same way after partitioned on a daily basis. We carry out three tasks that reveal the born and death, life expectancy, and longetivity distribution of UTXO in Kaggle.

###Task 1: UTXO born and death

In Task 1, we compute the number of UTXOs created and spent on each day. To see how many UTXOs were born in a day, we query the data from Bigquery that only keeps the UTXOs with block_timestamp within that day. We then sum the UTXOs in each transaction block to get the total UTXOs born in the day. Similarly, we query Bigquery again to keep the UTXOs with spent_block_timestamp within that day to get the total number of UTXOs spent on that date.

With these, it is possible to further compute the net new UTXOs in each day by minusing the number of UTXOs dead from the number of UTXOs born. The cumulative sum of net new UTXOs is the total number of UTXOs existing at each day.

> $$Net\_new[day = i] = Born[day = i] - Death[day = i] $$
> $$Cum[day = i] = \sum_{i \leq date} Net\_new[day = i]$$

In the kaggle notebook, Task1_born(data) and Task1_dead(data) are two functions defined to carry out this part of analysis.

###Task 2: UTXO weighted average life expectancy (WALE)

Weighted average life expectancy refers to the average length of living (from created to spent) of UTXOs that are spent before a given date, weighted by the value of UTXO. This shows how fast a UTXO is spent on average. To measure this, we query the data with spent_block_timestamp within each day to get the average life expectancy of UTXOs spent **on** that date.

> $$Life\_length = spent\_block\_timestamp - block\_timestamp$$
> $$WALE(day = i) = \sum_{day=i} (UTXO * Life\_length)/Death[day=i]$$

After calculating the WALE on each date, it is possible to calculate the WALE for the UTXOs spent **before** a given date:

> $$WALE(day\leq today)=\frac{\sum_{i \leq today}WALE(day=i)*Death[day=i]}{\sum_{i\leq today}Death[day=today]} $$

In the kaggle notebook, Task2(data) is defined to carry out this part of analysis.

###Task 3: UTXO Life Length Distribution

The life length of a UTXO is the difference between its spent_block_timestamp and its block_timestamp, i.e. the gap between the time created and the time spent. We would like to see the distribution of UTXOs' life length up to some date. To do this, we first compute the life length of each UTXO spent on a specific date and categorize them into the following categories: <1d, 1d-1m, 1m-1q, 1q-6m, 6m-1y, 1y-2y, 2y-3y, 3y-4y, 4y-5y, >5y. A categorical map is applied on life length to get categories. The total value of UTXOs in a category that are spent before some date can be computed by spent date:

> $$C_k[day \leq today] = \sum_{i\leq today} C_k[day = i] $$

In the kaggle notebook, Task3(data) is defined to carry out this part of analysis. cal(x) is the categorical map we use. Task3_process(result) is used to transform the analysis result to a form easy to record and visualize by expanding the distribution data.

#Visualizing Data in Google Colab

With the raw data processed and transformed to time series easy to visualize, several visualizations are created with the package plotly. The visualizations are made in the following Colab Notebook: [UTXO Visualizations](https://colab.research.google.com/drive/17B6YxJrVeI2qhTHAe5FaJxGqb92SR0jq?usp=sharing)